In [1]:
import os
import sys
import grid2op

res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

In [2]:
import numpy as np
from grid2op import make
from grid2op.Agent import RandomAgent
max_iter = 100
train_iter = 500
env_name = "rte_case14_redisp"
env = make(env_name,test=True)
env.seed(0)#这是为了保证环境中所有的随机源文件是可复制的
my_agent = RandomAgent(env.action_space)
my_agent.seed(0)#这是为了保证random agent所有的actions是一样的

D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\MakeEnv\Make.py:265: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [3]:
#从agent视角看到的action
for el in range(3):
    print(my_agent.my_act(None,None))

172
47
117


In [4]:
#从电网角度看到的action
for el in range(3):
    print(my_agent.act(None,None))

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element:
	 	 - assign bus 2 to line (extremity) 1 [on substation 4]
	 	 - assign bus 2 to line (extremity) 9 [on substation 4]
	 	 - assign bus 1 to line (extremity) 11 [on substation 4]
	 	 - assign bus 2 to line (origin) 17 [on substation 4]
	 	 - assign bus 2 to load 4 [on substation 4]
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT force any line status
	 - NOT switch any line status
	 - Change the bus of the following element:
	 	 - switch bus of line (origin) 11 [on substation 3]
	 	 - switch bus of line (origin) 15 [on substation 3]
	 - NOT force any particular bus configuration
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT f

In [5]:
print("To install l2rpn_baselines, either uncomment the cell below, or type, in a command prompt:\n{}".format(
    ("\t{} -m pip install l2rpn_baselines".format(sys.executable))))

To install l2rpn_baselines, either uncomment the cell below, or type, in a command prompt:
	D:\virtualenvironment\L2PRN_grid2op\python.exe -m pip install l2rpn_baselines


In [6]:
#tf2.0 friendly
import numpy as np
import random
import warnings
import l2rpn_baselines

In [7]:
#converters可以将observation类自动转化为向量，从数值映射actions来完成之间定义的aciton对象
#本质上，converter允许agent控制action space，这样一个agent就可以控制一个简单的、通用格式的action
#并能完成从简单结构到复杂action/observation的映射
from grid2op.Agent import AgentWithConverter
class MyAgent(AgentWithConverter):
    def __init__(self, action_space, action_space_converter=None):
        super(MyAgent, self).__init__(action_space=action_space, action_space_converter=action_space_converter)
        # for example you can define here all the actions you will consider
        self.my_actions = [action_space(),
                           action_space({"redispatching": [0,+1]}),
                           action_space({"set_line_status": [(0,-1)]}),
                           action_space({"change_bus": {"lines_or_id": [12]}}),
                          ...
                          ]
        # or load them from a file for example...
        # self.my_action = np.load("my_action_pre_selected.npy")

        # you can also in this agent load a neural network...
        self.my_nn_model = model.load("my_saved_neural_network_weights.h5")

    def convert_obs(self, observation):
        """
        This method is used to convert the observation, represented as a class Observation in input
        into a "transformed_observation" that will be manipulated by the agent
        An example here will transform the observation into a numpy array.

        It is recommended to modify it to suit your needs.

        """
        return observation.to_vect()

    def convert_act(self, encoded_act):
        """
        This method will take an "encoded_act" (for example a integer) into a valid grid2op action.
        """
        if encoded_act < 0 or encoded_act > len(self.my_action):
            raise RuntimeError("Invalid action with id {}".format(encoded_act))
        return self.my_actions[encoded_act]

    def my_act(self, transformed_observation, reward, done=False):
        """
        This is the main function where you can take your decision.

        Instead of:
        - calling "act(observation, reward, done)" you implement 
          "my_act(transformed_observation, reward, done)"
        - this manipulates only "transformed_observation" fully flexible as you defined "convert_obs"
        - and returns "encoded_action" that are then digest automatically by 
          "convert_act(encoded_act)" and to return valid actions.

        Here we suppose, as many dqn agent, that `my_nn_model` return a vector of size 
        nb_actions filled with number between 0 and 1 and we take the action given the highest score
        """
        pred_score = self.my_nn_model.predict(transformed_observation, reward, done)
        res = np.argmax(pred_score)
        return res

In [7]:
from l2rpn_baselines.utils import BaseDeepQ, TrainingParam
import tensorflow.keras as tfk
class DuelQ_NN(BaseDeepQ):
    """Constructs the desired deep q learning network"""
    def __init__(self,
                 nn_params,  # neural network meta parameters, defining its architecture
                 training_param=None  # training scheme (learning rate, learning rate decay, etc.)
                ):
        self.action_size = action_size
        self.observation_size = observation_size
        HIDDEN_FOR_SIMPLICITY

    def construct_q_network(self):
        """
        It uses the architecture defined in the `nn_archi` attributes.

        """
        self.model = Sequential()
        input_layer = Input(shape=(self.nn_archi.observation_size,),
                            name="observation")

        lay = input_layer
        for lay_num, (size, act) in enumerate(zip(self.nn_archi.sizes, self.nn_archi.activs)):
            lay = Dense(size, name="layer_{}".format(lay_num))(lay)  # put at self.action_size
            lay = Activation(act)(lay)

        fc1 = Dense(self.action_size)(lay)
        advantage = Dense(self.action_size, name="advantage")(fc1)

        fc2 = Dense(self.action_size)(lay)
        value = Dense(1, name="value")(fc2)

        meaner = Lambda(lambda x: K.mean(x, axis=1))
        mn_ = meaner(advantage)
        tmp = subtract([advantage, mn_])
        policy = add([tmp, value], name="policy")

        self.model = Model(inputs=[input_layer], outputs=[policy])
        self.schedule_model, self.optimizer_model = self.make_optimiser()
        self.model.compile(loss='mse', optimizer=self.optimizer_model)

        self.target_model = Model(inputs=[input_layer], outputs=[policy])
    def predict_movement(self, data, epsilon, batch_size=None):
        """
        Predict movement of game controler where is epsilon
        probability randomly move.

        This was part of the BaseDeepQ and was copy pasted without any modifications
        """
        if batch_size is None:
            batch_size = data.shape[0]

        rand_val = np.random.random(batch_size)
        q_actions = self.model.predict(data, batch_size=batch_size)

        opt_policy = np.argmax(np.abs(q_actions), axis=-1)
        opt_policy[rand_val < epsilon] = np.random.randint(0, self.action_size, size=(np.sum(rand_val < epsilon)))
        return opt_policy, q_actions[0, opt_policy]

In [9]:
from grid2op.Agent import AgentWithConverter  # all converter agent should inherit this
from grid2op.Converter import IdToAct  # this is the automatic converter to convert action given as ID (integer)
# to valid grid2op action (in particular it is able to compute all actions).

from l2rpn_baselines.utils import DeepQAgent
from l2rpn_baselines.DoubleDuelingDQN.DoubleDuelingDQN_NN import DoubleDuelingDQN_NN
class DuelQSimple(DeepQAgent):
    def __init__(self,
                 action_space,
                 nn_archi,
                 HIDDEN_FOR_SIMPLICITY
                ):
        ...
        HIDDEN_FOR_SIMPLICITY
        ...
        # Load network graph
        self.deep_q  = None ## is loaded when first called or when explicitly loaded

    # Agent Interface
    def convert_obs(self, observation):
        """
        Generic way to convert an observation. This transform it to a vector and the select the attributes
        that were selected in :attr:`l2rpn_baselines.utils.NNParams.list_attr_obs` (that have been
        extracted once and for all in the :attr:`DeepQAgent._indx_obs` vector).
        """
        obs_as_vect = observation.to_vect()
        self._tmp_obs[:] = obs_as_vect[self._indx_obs]
        return self._tmp_obs
    
    def convert_act(self, action):
        """
        calling the convert_act method of the base class.
        This is not mandatory as this is the standard behaviour in OOP (object oriented programming)
        We only show it here as illustration.
        """
        return super().convert_act(action)

    def my_act(self, transformed_observation, reward, done=False):
        """
        This function will return the action (its id) selected by the underlying 
        :attr:`DeepQAgent.deep_q` network.


        Before being used, this method require that the :attr:`DeepQAgent.deep_q` is created. 
        To that end a call to :func:`DeepQAgent.init_deep_q` needs to have been performed 
        (this is automatically done if you use baseline we provide and their `evaluate` 
        and `train` scripts).
        """
        predict_movement_int, *_ = self.deep_q.predict_movement(transformed_observation, epsilon=0.0)
        res = int(predict_movement_int)
        self._store_action_played(res)
        return res

In [34]:
#训练模型
# create an environment
env = make(env_name,test=True) 
# don't forget to set "test=False" (or remove it, as False is the default value) for "real" training

# import the train function and train your agent
from l2rpn_baselines.DuelQSimple import train
from l2rpn_baselines.utils import NNParam, TrainingParam
agent_name = "test_agent"
save_path = "saved_agent_DDDQN_{}".format(train_iter)
logs_dir="tf_logs_DDDQN"


# we then define the neural network we want to make (you may change this at will)
## 1. first we choose what "part" of the observation we want as input, 
## here for example only the generator and load information
## see https://grid2op.readthedocs.io/en/latest/observation.html#main-observation-attributes
## for the detailed about all the observation attributes you want to have
li_attr_obs_X = ["prod_p", "prod_v", "load_p", "load_q"]
# this automatically computes the size of the resulting vector
observation_size = NNParam.get_obs_size(env, li_attr_obs_X) 


D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\MakeEnv\Make.py:265: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [36]:
## 2. then we define its architecture
sizes = [300, 300, 300]  # 3 hidden layers, of 300 units each, why not...
activs =  ["relu" for _ in sizes]  # all followed by relu activation, because... why not
## 4. you put it all on a dictionnary like that (specific to this baseline)
kwargs_archi = {'observation_size': observation_size,
                'sizes': sizes,
                'activs': activs,
                "list_attr_obs": li_attr_obs_X}

# you can also change the training parameters you are using
# more information at https://l2rpn-baselines.readthedocs.io/en/latest/utils.html#l2rpn_baselines.utils.TrainingParam
tp = TrainingParam()
tp.batch_size = 32  # for example...
tp.update_tensorboard_freq = int(train_iter / 10)
tp.save_model_each = int(train_iter / 3)
tp.min_observation = int(train_iter / 5)
train(env,
      name=agent_name,
      iterations=train_iter,
      save_path=save_path,
      load_path=None, # put something else if you want to reload an agent instead of creating a new one
      logs_dir=logs_dir,
      kwargs_archi=kwargs_archi,
      training_param=tp)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:17<00:00, 28.34it/s]


In [10]:
save_path

'saved_agent_DDDQN_500'

In [37]:
from grid2op.Runner import Runner
from tqdm.notebook import tqdm
# chose a scoring function (might be different from the reward you use to train your agent)
from grid2op.Reward import L2RPNReward
scoring_function = L2RPNReward 
path_save_results = "{}_results".format(save_path)

# load your agent (a bit technical to know exactly what to import and how to use it)
# this is why we made the "evaluate" function that simplifies greatly the process.
from l2rpn_baselines.DuelQSimple import DuelQSimple
from l2rpn_baselines.DuelQSimple import DuelQ_NNParam
path_model, path_target_model = DuelQ_NNParam.get_path_model(save_path, agent_name)
nn_archi = DuelQ_NNParam.from_json(os.path.join(path_model, "nn_architecture.json"))

my_agent = DuelQSimple(env.action_space, nn_archi, name=agent_name)
my_agent.load(save_path)
my_agent.init_obs_extraction(env)

# here we do that to limit the time take, and will only assess the performance on "max_iter" iteration
dict_params = env.get_params_for_runner()
dict_params["gridStateclass_kwargs"]["max_iter"] =  max_iter
# make a runner from an intialized environment
runner = Runner(**dict_params, agentClass=None, agentInstance=my_agent)

# run the episode
res = runner.run(nb_episode=2, path_save=path_save_results, pbar=tqdm)
print("The results for the trained agent are:")
for _, chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - total score: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

AttributeError: 'Environment_rte_case14_redisp' object has no attribute 'get_indx_extract'

In [23]:
from l2rpn_baselines.DuelQSimple import evaluate
path_save_results = "{}_results".format(save_path)

evaluated_agent, res_runner = evaluate(env,
                                       name=agent_name,
                                       load_path=save_path,
                                       logs_path=path_save_results,
                                       nb_episode=2,
                                       nb_process=1,
                                       max_steps=100,
                                       verbose=True,
                                       save_gif=False)

episode:   0%|                                                                                   | 0/2 [00:00<?, ?it/s]

Model: "functional_65"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation (InputLayer)        [(None, 32)]         0                                            
__________________________________________________________________________________________________
layer_0 (Dense)                 (None, 300)          9900        observation[0][0]                
__________________________________________________________________________________________________
activation_48 (Activation)      (None, 300)          0           layer_0[0][0]                    
__________________________________________________________________________________________________
layer_1 (Dense)                 (None, 300)          90300       activation_48[0][0]              
______________________________________________________________________________________


episode:   6%|████▍                                                                    | 6/100 [00:00<00:01, 53.72it/s]

INFO: "Creating path "C:\code\python\jupyter\Grid2Op\saved_agent_DDDQN_500_results\0" to save the episode 0"



episode:  50%|█████████████████████████████████████▌                                     | 1/2 [00:02<00:02,  2.71s/it]

INFO: "Env: 1.62s
	 - apply act 0.31s
	 - run pf: 1.27s
	 - env update + observation: 0.03s
Agent: 0.20s
Total time: 1.89s
Cumulative reward: 121379.789062"



episode:   6%|████▍                                                                    | 6/100 [00:00<00:01, 53.70it/s]

INFO: "Creating path "C:\code\python\jupyter\Grid2Op\saved_agent_DDDQN_500_results\1" to save the episode 1"



episode: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.44s/it]

INFO: "Env: 1.59s
	 - apply act 0.31s
	 - run pf: 1.25s
	 - env update + observation: 0.04s
Agent: 0.19s
Total time: 1.84s
Cumulative reward: 123131.984375"
Evaluation summary:
chronics at: 0	total score: 121379.789062	time steps: 100/100
chronics at: 1	total score: 123131.984375	time steps: 100/100
The agent played 1 different action
Action with ID 118 was played 200 times
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT force any line status
	 - switch status of 1 powerlines ([17])
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration
-----------


AttributeError: 'Environment_rte_case14_redisp' object has no attribute 'get_indx_extract'